In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.optim as optim

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Preprocessing

In [3]:
# Load Data
df1 = pd.read_csv('data/ferm.csv', index_col=0, low_memory=False)
df2 = pd.read_csv('data/multiome_all.csv', index_col=0, low_memory=False)

In [16]:
# Merge Datasets
df = df1.join(df2, lsuffix='_ferm', rsuffix='_resp')

In [5]:
# Drop rows with missing values
cln = df.dropna()

In [6]:
# Separate datasets by molecule type
protein = cln.loc[cln['Molecule Type_ferm'] == 'Protein'].drop(columns=['Molecule Type_ferm', 'Molecule Type_resp'])
metabolite = cln.loc[cln['Molecule Type_ferm'] == 'Metabolite'].drop(columns=['Molecule Type_ferm', 'Molecule Type_resp'])
lipid = cln.loc[cln['Molecule Type_ferm'] == 'Lipid'].drop(columns=['Molecule Type_ferm', 'Molecule Type_resp'])

In [7]:
# Impute missing values using KNN
imputer = KNNImputer(n_neighbors=2)

In [8]:
# Impute proteins
pfi = imputer.fit_transform(protein)
protein_df = pd.DataFrame(data=pfi.T, index=protein.T.index, columns=protein.T.columns)

In [9]:
# Impute metabolites
mfi = imputer.fit_transform(metabolite)
metabolite_df = pd.DataFrame(data=mfi.T, index=metabolite.T.index, columns=metabolite.T.columns)

In [10]:
# Impute lipids
lfi = imputer.fit_transform(lipid)
lipid_df = pd.DataFrame(data=lfi.T, index=lipid.T.index, columns=lipid.T.columns)

In [11]:
# Combine Imputed Datasets
combined_df = protein_df.join([metabolite_df, lipid_df])

# Construct Graph

In [12]:
# Convert combined DataFrame to tensor
features = torch.tensor(combined_df.values, dtype=torch.float)

In [13]:
# Example: Create a fully connected graph
edge_index = torch.tensor([[i, j] for i in range(features.shape[0]) for j in range(features.shape[0]) if i != j], dtype=torch.long).t().contiguous()

In [14]:
# Create PyTorch Geometric data object
data = Data(x=features, edge_index=edge_index)

# Define Graph Autoencoder

In [15]:
class GraphAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphAutoencoder, self).__init__()
        self.encoder = GCNConv(input_dim, hidden_dim)
        self.decoder = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        hidden = F.relu(self.encoder(x, edge_index))
        output = self.decoder(hidden, edge_index)
        return output

input_dim = features.shape[1]
hidden_dim = 64
output_dim = input_dim
model = GraphAutoencoder(input_dim, hidden_dim, output_dim)

# Training setup
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train(data, model, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = criterion(output, data.x)
    loss.backward()
    optimizer.step()
    return loss.item()

# Training loop
epochs = 100
for epoch in range(epochs):
    loss = train(data, model, optimizer, criterion)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

print('Training completed')

# Save the model
torch.save(model.state_dict(), 'graph_autoencoder.pth')

Epoch 0, Loss: 0.1420
Epoch 10, Loss: 0.1304
Epoch 20, Loss: 0.1292
Epoch 30, Loss: 0.1287
Epoch 40, Loss: 0.1285
Epoch 50, Loss: 0.1285
Epoch 60, Loss: 0.1285
Epoch 70, Loss: 0.1285
Epoch 80, Loss: 0.1285
Epoch 90, Loss: 0.1285
Training completed
